# Dados faltantes


**Notebook destinado ao estudo de técnicas para lidar com dados faltantes** 

###  Breve explicação sobre o tema:



Um dos problemas mais comuns que surgem durante as análises de dados, é a presença de valores faltantes. Não ha uma fórmula fechada que nos mostre como tratar a ausência de valores. Existem diversas abordagens e cada uma delas será mais interessante de acordo com os dados e objetivos da análise.

O primeiro passo para entender e pensar em estratégias para lidar com missing data é entender o motivo desta ausência, podemos não ter as informações por diferentes motivos, e as algumas vezes, a ausência do dado é uma informação, portanto, antes de pensar em como tratar estes casos precisamos entender os tipos de dados faltantes com os quais podemos esbarrar:

**Tipos de dados faltantes**


* **MCAR** - Missing Completely Random

Dados perdidos de forma completamente aleatória. Neste caso não há justificativa para a ocorrência, o fato do valor estar ausente não tem nada a ver com seu valor ou com outras variáveis.  Como exemplo para este caso, podemos citar dados perdidos durante integração entre sistemas ou na imputação dos dados.

* **MAR** - Missing at Random

Quando temos dados perdidos de forma aleatória, que dizer que a ausência do valor não tem a ver com o valor em sí, não se sabe se o dado deveria existir ou não. Nestes casos os dados não estão faltantes em todas as observações, geralmente apenas em alguns casos. Sendo assim, pode-se usar os dados observados parar fazer alguma análise preditiva e imputar os valores ausentes.
 Como exemplo deste tipo de dados faltantes, podemos pensar num questionário onde alguém não viu o campo de idade e o deixou em branco. É possível que com os outros campos preenchidos, possamos estimar a idade deste indivíduo.

* **MNAR** - Missing not at Random

Pode ser também que a ausência da informação em sí, seja uma informação. Nestes casos a falta de informação pode ter algum motivo especifico. O motivo pode ter a ver com o valor em sí ( Por exemplo, uma pessoa com a renda muito alta não querer revelar este valor, ou pessoas LGBTQIA+ não estarem confortáveis em responder sobre sua sexualidade), ou pode ter a ver com alguma outra causa. Suponhamos que mulheres entre 45 e 55  anos não responderam a uma pergunta específica. 
Nestes casos os dados faltantes podem trazer alguma informação mesmo que não seja de forma direta. Se removermos estes casos, podemos viciar o nosso grupo de dados. No exemplo das mulheres entre 44 e 55 anos, teríamos uma base que não seria capaz de representar tal grupo.


Após entendermos um pouco dos tipos de dados faltantes, podemos olhar para cada caso e traçar a estratégia a ser utilizada. Não podemos definir qual a estratégia é ideal para cada caso, mas devemos tentar entender a estrutura desta ausência de dados e traçar a estratégia que parecer melhor se encaixar no problema, as estratégia vão desde eliminar os valores faltantes até técnicas avançadas para prever os valores faltantes.

Vamos explorar a seguir algumas das possíveis maneiras de lidar com estes problemas, mas exitem N outros métodos que podem ser usados.


## Carregando pacotes 

In [2]:
import pandas as pd
import numpy as np
#from sklearn.model_selection import train_test_split

### Algumas configurações padrão
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

### Baixar os dados do kaggle
from kaggle.api.kaggle_api_extended import KaggleApi
## Extrair arquivos Zip
from zipfile import ZipFile
## Listagem dos diretórios
from os import listdir

## Base de dados

**Explicar aqui a base de dados**

### Lendo a base do Kaggle

Abrindo a conexão com o Kaggle, que ja foi previamente configurada. Mas detalhes sobre a conexão direta com o Kaggle podem ser obtidos em: https://python.plainenglish.io/how-to-use-the-kaggle-api-in-python-4d4c812c39c7

In [3]:
api = KaggleApi()
api.authenticate()

O primeiro passo é baixar o banco de dados. Neste caso optei por baixar todos os arquivos disponíveis.

In [4]:
api.dataset_download_files('anthonypino/melbourne-housing-market')

Após baixar os aquivos, vamos verificar no noss diretorio para saber qual arquivo extrair

In [5]:
listdir()

['.ipynb_checkpoints',
 '.Rhistory',
 'data',
 'melbourne-housing-market.zip',
 '[Python]DadosFaltantes.ipynb',
 '[R]DadosFaltantes.Rmd']

O arquivo baixado foi 'melbourne-housing-market.zip', vamso extrai-lo para dentro da pasta 'data'

In [6]:
zf = ZipFile('melbourne-housing-market.zip')
# Criando uma pasta chadama data e salvando os dados nela
zf.extractall('data/') 
zf.close()
listdir('data/')

['MELBOURNE_HOUSE_PRICES_LESS.csv',
 'Melbourne_housing_FULL.csv',
 'NFL Play by Play 2009-2016 (v3).csv',
 'NFL Play by Play 2009-2017 (v4).csv',
 'NFL Play by Play 2009-2018 (v5).csv']

Após a extarção dos dados, vamos carregar o arquivo desejado.

## Lendo os dados

In [11]:
df = pd.read_csv('data/Melbourne_housing_FULL.csv')
print('Base de dados importada com sucesso! \n{} linhas e {} colunas'.format(df.shape[0], df.shape[1]))

Base de dados importada com sucesso! 
34857 linhas e 21 colunas


Inspecionando as primeiras e ultimas linhas da tabela carregada

In [12]:
df.head()

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,68 Studley St,2,h,NaN,SS,Jellis,3/09/2016,2.50,3067.00,2.00,1.00,1.00,126.00,NaN,NaN,Yarra City Council,-37.80,145.00,Northern Metropolitan,4019.00
1,Abbotsford,85 Turner St,2,h,1480000.00,S,Biggin,3/12/2016,2.50,3067.00,2.00,1.00,1.00,202.00,NaN,NaN,Yarra City Council,-37.80,145.00,Northern Metropolitan,4019.00
2,Abbotsford,25 Bloomburg St,2,h,1035000.00,S,Biggin,4/02/2016,2.50,3067.00,2.00,1.00,0.00,156.00,79.00,1900.00,Yarra City Council,-37.81,144.99,Northern Metropolitan,4019.00
3,Abbotsford,18/659 Victoria St,3,u,NaN,VB,Rounds,4/02/2016,2.50,3067.00,3.00,2.00,1.00,0.00,NaN,NaN,Yarra City Council,-37.81,145.01,Northern Metropolitan,4019.00
4,Abbotsford,5 Charles St,3,h,1465000.00,SP,Biggin,4/03/2017,2.50,3067.00,3.00,2.00,0.00,134.00,150.00,1900.00,Yarra City Council,-37.81,144.99,Northern Metropolitan,4019.00


In [13]:
df.tail()

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
34852,Yarraville,13 Burns St,4,h,1480000.00,PI,Jas,24/02/2018,6.30,3013.00,4.00,1.00,3.00,593.00,NaN,NaN,Maribyrnong City Council,-37.81,144.88,Western Metropolitan,6543.00
34853,Yarraville,29A Murray St,2,h,888000.00,SP,Sweeney,24/02/2018,6.30,3013.00,2.00,2.00,1.00,98.00,104.00,2018.00,Maribyrnong City Council,-37.82,144.89,Western Metropolitan,6543.00
34854,Yarraville,147A Severn St,2,t,705000.00,S,Jas,24/02/2018,6.30,3013.00,2.00,1.00,2.00,220.00,120.00,2000.00,Maribyrnong City Council,-37.82,144.88,Western Metropolitan,6543.00
34855,Yarraville,12/37 Stephen St,3,h,1140000.00,SP,hockingstuart,24/02/2018,6.30,3013.00,NaN,NaN,NaN,NaN,NaN,NaN,Maribyrnong City Council,NaN,NaN,Western Metropolitan,6543.00
34856,Yarraville,3 Tarrengower St,2,h,1020000.00,PI,RW,24/02/2018,6.30,3013.00,2.00,1.00,0.00,250.00,103.00,1930.00,Maribyrnong City Council,-37.82,144.89,Western Metropolitan,6543.00


## Entendendo os valores faltante

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34857 entries, 0 to 34856
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Suburb         34857 non-null  object 
 1   Address        34857 non-null  object 
 2   Rooms          34857 non-null  int64  
 3   Type           34857 non-null  object 
 4   Price          27247 non-null  float64
 5   Method         34857 non-null  object 
 6   SellerG        34857 non-null  object 
 7   Date           34857 non-null  object 
 8   Distance       34856 non-null  float64
 9   Postcode       34856 non-null  float64
 10  Bedroom2       26640 non-null  float64
 11  Bathroom       26631 non-null  float64
 12  Car            26129 non-null  float64
 13  Landsize       23047 non-null  float64
 14  BuildingArea   13742 non-null  float64
 15  YearBuilt      15551 non-null  float64
 16  CouncilArea    34854 non-null  object 
 17  Lattitude      26881 non-null  float64
 18  Longti

In [15]:
df.isnull().sum(axis = 0)

Suburb               0
Address              0
Rooms                0
Type                 0
Price             7610
Method               0
SellerG              0
Date                 0
Distance             1
Postcode             1
Bedroom2          8217
Bathroom          8226
Car               8728
Landsize         11810
BuildingArea     21115
YearBuilt        19306
CouncilArea          3
Lattitude         7976
Longtitude        7976
Regionname           3
Propertycount        3
dtype: int64

Podemos observar varias colunas com valores faltantes, e cada coluna pode ter um tipo diferente de dado faltante. 
Vamos utilizar conhecimentos anteiores para definir o tipo de dado faltante em cada coluna, para em seguida traçar as estratégias de tratamento.

Versão python 3.8.8